In [51]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
from sklearn import preprocessing, cross_validation
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [141]:
data_path = '../../data/card-detection/features/features.csv'
print("TensorFlow version: {}".format(tf.VERSION))

pd.options.display.max_colwidth = 1000

COLUMN_NAMES = ['data','hasFace','image_contour', 'class']

CLASSES = ['driving license', 'financial card', 'text']

NUM_CLASSES = len(CLASSES)
MAX_VOCABULARY_SIZE = 500
BATCH_SIZE = 10
EPOCHS = 100
LEARNING_RATE = 0.001


def raw_data_fn(y_name='class'):
    dataset = pd.read_csv(data_path, encoding='utf-8')
    
    #drop all df with value NaN
    dataset = dataset.dropna(subset=['data', 'hasFace', 'image_contour', 'class'])
    TRAINING_SIZE = len(dataset)
    #TEST_SIZE = len(dataset)
    train_images = np.empty((TRAINING_SIZE,600,1050), np.float32)
    
    image_path = dataset['image_contour']
    i=0
    for img in image_path:
        x = load_img(img)
        x = img_to_array(x)
        x = x[:,:,0]
        train_images[i,:,:] = x
        i+=1
        
    # Reshape from (N, 28, 28) to (N, 784)
    train_images = np.reshape(train_images, (TRAINING_SIZE, 630000))
    train_images /= 255  #shape:16*6300000
   
    # text data preparation 
    description = dataset['data'].astype(str)    #shape:16*500
    description.resize(16, 6300000)
    print(description.shape)
    
    #get labels 
    classes = dataset['class']

    # define Tokenizer with Vocab Size
    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=MAX_VOCABULARY_SIZE)
    tokenizer.fit_on_texts(description)
    
    # define encoder and make labels
    encoder = preprocessing.LabelBinarizer()
    encoder.fit(CLASSES)
    
    #make features and labels
    labels = encoder.transform(classes)
    features = tokenizer.texts_to_matrix(description, mode='tfidf')
    #print(features.shape)
    
    #prepare boolean data
    hasFace = dataset["hasFace"]   #shape:16*1
    #return (features, labels, tokenizer)
    
raw_data_fn()
# features, labels, tokenizer = raw_data_fn()
# #print(features[0])
# #print(labels)

# def load_data_fn(features, labels, test_size=0.3):
#     return cross_validation.train_test_split(features, labels, test_size=test_size, random_state=42)

# def model_fn():
#     model = tf.keras.Sequential()
#     model.add(tf.keras.layers.Dense(512, activation=tf.nn.relu, input_shape=(MAX_VOCABULARY_SIZE,)))
#     model.add(tf.keras.layers.Dense(NUM_CLASSES, activation=tf.nn.softmax))

#     # Create a TensorFlow optimizer, rather than using the Keras version
#     # This is currently necessary when working in eager mode
#     optimizer = tf.train.RMSPropOptimizer(learning_rate=0.001)

#     # We will now compile and print out a summary of our model
#     model.compile(loss='categorical_crossentropy',
#                   optimizer=optimizer,
#                   metrics=['accuracy'])

#     model.summary()
#     return model


# train_features, test_features, train_labels, test_labels = load_data_fn(features, labels)
# print("Train count {features/labels}: %s/%s" % (len(train_features), len(train_labels)))
# print("Test count {features/labels}: %s/%s" % (len(test_features), len(test_labels)))

# #create a model for training
# model = model_fn()

# #train the training set with model
# model.fit(train_features, train_labels, epochs=EPOCHS)

# #evaluate the model on test set
# loss, accuracy = model.evaluate(test_features, test_labels)
# print("Loss : %s   Accuracy: %s" % (loss, accuracy))

TensorFlow version: 1.8.0


AttributeError: 'Series' object has no attribute 'resize'

In [120]:
# gImg = cv2.imread(info['image_contour']) 
# temp_t = tf.image.convert_image_dtype(
#     gImg,
#     tf.float32
# )

def predict_fn(model, labels, tokenizer, description):
    name_description = description

    feature = tokenizer.texts_to_matrix(np.array([name_description]), mode='tfidf')
    prediction = model.predict(feature)
    
    predicted_label = CLASSES[np.argmax(prediction[0])]
    return predicted_label

In [ ]:
description1 = "Text messaging, or texting, is the act of composing and sending electronic messages, typically consisting of alphabetic and numeric characters, between two or more users of mobile phones, tablets, desktops/laptops, or other devices."
description2 = 'LICENCE 1 STANLEY 2 JOE OLIVIA 3 11 03 1976 UNITED KINGDOM 4a  MORGA7531163M9 BURNS CRESCENT EDINBURGH EH1 QGP 9 k l n p q k 7mm rl DVLA INTERNAL USAGE'
description3 = 'random gibberish text which is neither a driving license or a financial card 347437493 290422 BHHDS *()&^'

predicted_label = predict_fn(model, labels, tokenizer, description2)
print("description: %s \npredicted class: %s" % (description2, predicted_label))